# Nextclade comparison 
1. Select a subset of sequences not used in train/val/test for the 8 clades the model was trained on 
    - [x] consider the data in datasets.json
    - [x] drop those files from the GISAID metadata in  /home/disco/Github/GISAID/metadata_tsv_2021_11_11/metadata.tsv
    - [x] then, select new files from the metadata for the 8 clades
2. Select a new `global-test` dataset
    - [x] Extract all the selected sequences in individual fasta files to evaluate the model
    - [x] Also, write all the extracted sequences in a single fasta file to run nextclade
3. Compare results
    - map all predictions to the same nomenclature (GISAID). 
    - The output from Nextclade is a column called `clade` corresponding to one of the `Nextrain Clade` that can be found in [this](https://covariants.org/variants) table.
    - To compare results, map the `Nextrain Clade` (nextclade output) to the `Pangolineage`, that can be found also in the GISAID metadata for each one of the selected sequences.
    - Once all predictions are in the GISAID nomenclature, proceed with a comparison of results. 

In [1]:
import json 
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
with open("datasets.json","r") as fp: 
    datasets = json.load(fp)
    
npy_files = []
for list_set in datasets.values():
    npy_files.extend(list_set)
    
len(npy_files) 

50718

In [3]:
# Clean dataset
npy_files = [file for file in  npy_files if "(1)" not in file]

In [4]:
len(npy_files), npy_files[0]

(44073,
 'npy-8-mer/hCoV-19/V/hCoV-19_England_SHEF-CE129_2020|2020-03-22|2020-04-29.npy')

In [5]:
npy2fastaid = lambda npy: npy.split("/")[-1].replace(".npy",".fasta").replace("_","/")

In [17]:
# list of fastaid used in train/val/test
fasta_id_used = [npy2fastaid(file) for file in npy_files]

# load metadata

In [7]:
CLADES = ['S','L','G','V','GR','GH','GV','GK']#,'GRY']
COLS = ["Virus name", "Accession ID", "Collection date", "Submission date","Clade", "Host", "Is complete?"]

In [8]:
path_metadata = "/home/disco/Github/GISAID/metadata_tsv_2021_11_11/metadata.tsv"
metadata = pd.read_csv(path_metadata, sep="\t", usecols=COLS)

In [9]:
metadata.shape

(5045607, 7)

In [10]:
# Remove NaN in Clades and not-complete sequences
metadata.dropna(axis="rows",
            how="any",
            subset=["Is complete?", "Clade"], 
            inplace=True,
            )

In [11]:
metadata.shape

(4956947, 7)

In [12]:
# Filter by Clades and Host
CLADES = tuple(clade for clade in CLADES)
metadata.query(f"`Clade` in {CLADES} and `Host`=='Human'", inplace=True)

In [13]:
metadata.shape

(3986669, 7)

In [14]:
# Generate id of sequences in fasta file: "Virus name|Accession ID|Collection date"
metadata["fasta_id"] = metadata.progress_apply(lambda row: "|".join([row["Virus name"],row["Collection date"],row["Submission date"]]), axis=1)

100%|████████████████████████████████| 3986669/3986669 [01:01<00:00, 65119.95it/s]


In [23]:
import random
from collections import namedtuple
# subsample 
SAMPLES_PER_CLADE = 5000
SampleClade = namedtuple("SampleClade", ["fasta_id","clade"])
list_fasta_selected = []
for clade in tqdm(CLADES):
    samples_clade = metadata.query(f"`Clade` == '{clade}'")["fasta_id"].tolist()
    samples_clade = [sample for sample in samples_clade if sample not in fasta_id_used]
    random.shuffle(samples_clade)
    # select 'SAMPLES_PER_CLADE' samples for each clade, or all of them if available samples are less than required
    list_fasta_selected.extend([SampleClade(fasta_id, clade) for fasta_id in samples_clade[:SAMPLES_PER_CLADE]])

100%|██████████████████████████████████████████████| 8/8 [49:51<00:00, 373.97s/it]


In [18]:
metadata["fasta_id"][0] in fasta_id_used

False

In [25]:
from pathlib import Path
Path("nextclade-comparison").mkdir(exist_ok=True, parents=True)
pd.DataFrame(list_fasta_selected).to_csv("nextclade-comparison/undersample_by_clade.csv")

In [26]:
pd.DataFrame(list_fasta_selected).groupby("clade").size()

clade
G     5000
GH    5000
GK    5000
GR    5000
GV    5000
L     5000
S     5000
V     5000
dtype: int64

___
# Compare results
- https://covariants.org/variants
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8184080/ supplementary data contains a table (for the indian sequences) with all the linages (Nextclade, GISAID, Pangolin)
- https://cov-lineages.org/lineage_list.html only pangolin

### Relation between lineages are saved in dictionaries at `/master-lineages`: 
- **nextclade2pango**: obtained from covariants.orig/variants
- **pango2gisaid**: obtained from the metadata of GISAID

In [13]:
import json
import pandas as pd
pd.set_option("display.max_columns", None)

In [46]:
# load dictionaries from /master-lineages
with open("master-lineages/nextclade2pango.json","r") as fp:
    nextclade2pango = json.load(fp) # keys are list of pango clades

with open("master-lineages/pango2gisaid.json","r") as fp:
    pango2gisaid = json.load(fp) # keys are individual gisaid clades
    
def nextclade2gisaid(nextclade):
    list_pango = nextclade2pango.get(nextclade, ["not found"])
    if "not found" in list_pango:
        return "not found"
    # map pango lineages to gisaid clades
    list_gisaid = []
    for pango in list_pango: 
        list_gisaid.append(pango2gisaid.get(pango))
        
    return ",".join(set(list_gisaid))

In [29]:
# nextclade predictions
path_nextclade_preds = "/home/disco/Github/nextclade/output-comparison-nextclade/nextclade.tsv"
nextclade_preds = pd.read_csv(path_nextclade_preds, sep="\t", usecols=["seqName","clade"])
nextclade_preds.rename({'clade': 'nextclade'}, axis=1, inplace=True) 

In [30]:
nextclade_preds.head()

,seqName,nextclade
0,hCoV-19/Angola/CERI-KRISP-K019085/2021|2021-03...,21B (Kappa)
1,hCoV-19/Argentina/INEI018119/2020|2020-06-11|2...,20A
2,hCoV-19/Australia/ACT0179/2021|2021-08-19|2021...,21J (Delta)
3,hCoV-19/Australia/NSW-ICPMR-11015/2021|2021-10...,21J (Delta)
4,hCoV-19/Australia/NSW-ICPMR-11682/2021|2021-10...,21J (Delta)


In [53]:
path_model_preds = "results_nextclade_comparison.csv"
model_preds = pd.read_csv(path_model_preds)

# get ground truth for the sequence
model_preds["ground_truth"] = model_preds["path_fasta"].apply(lambda path: path.split("/")[2])

# extract seqName to merge with nextclade predictions
model_preds["seqName"] = model_preds["path_fasta"].apply(lambda path: path.split("/")[-1].replace("_","/")[:-6])

In [54]:
seqs_in_common = len(set(model_preds.seqName).intersection(set(nextclade_preds.seqName)))
print(f"There are {seqs_in_common} sequences shared between nextclade and the model")
print(f"Preds in nextclade {nextclade_preds.shape[0]}")
print(f"Preds in model {model_preds.shape[0]}")

There are 36491 sequences shared between nextclade and the model
Preds in nextclade 38749
Preds in model 38749


In [55]:
#list(set(model_preds.seqName) - set(nextclade_preds.seqName))[:10]

In [56]:
#list(set(nextclade_preds.seqName)-set(model_preds.seqName))[:10]

In [57]:
# merge predictions
preds = model_preds.merge(nextclade_preds, how="inner", on="seqName")

In [58]:
preds.head()

,Unnamed: 0,path_fasta,prob,pred_class,ground_truth,seqName,nextclade
0,0,data-nextclade-comparison/hCoV-19/GK/hCoV-19_U...,0.899028,GK,GK,hCoV-19/USA/MD-CDC-LC0260562/2021|2021-09-08|2...,21J (Delta)
1,1,data-nextclade-comparison/hCoV-19/GK/hCoV-19_A...,0.874016,GK,GK,hCoV-19/Aruba/AW-RIVM-52828/2021|2021-08-10|20...,21J (Delta)
2,2,data-nextclade-comparison/hCoV-19/GK/hCoV-19_E...,0.774449,GK,GK,hCoV-19/England/MILK-1C6044F/2021|2021-08-22|2...,21J (Delta)
3,3,data-nextclade-comparison/hCoV-19/GK/hCoV-19_F...,0.827125,GK,GK,hCoV-19/France/NAQ-CERBAHC-0717768/2021|2021|2...,21J (Delta)
4,4,data-nextclade-comparison/hCoV-19/GK/hCoV-19_A...,0.879893,GK,GK,hCoV-19/Aruba/AW-RIVM-51660/2021|2021-08-08|20...,21J (Delta)


In [59]:
# map nextclade prediction to gisaid
preds["nextclade2gisaid"] = preds["nextclade"].apply(lambda pred: nextclade2gisaid(pred))

In [60]:
preds.groupby(["nextclade2gisaid"]).size()

nextclade2gisaid
G,GR           630
GK            7791
GK,O          5604
GRY            270
GV             647
O             1064
not found    20485
dtype: int64

In [62]:
preds.groupby(["ground_truth","nextclade2gisaid"]).size()

ground_truth  nextclade2gisaid
G             G,GR                   7
              GK                   525
              GK,O                2753
              GRY                  103
              GV                    90
              O                     26
              not found           1066
GH            G,GR                 623
              GK                   275
              GK,O                 911
              GV                   482
              O                   1036
              not found           1207
GK            GK,O                   8
              GRY                  167
              not found           4488
GR            GK                  2271
              GK,O                1912
              GV                    75
              not found            219
GV            GK                  4720
              GK,O                  15
              not found            139
L             GK,O                   2
              not found          

___
### Metrics model

In [66]:
from collections import namedtuple
from sklearn.metrics import precision_recall_fscore_support

In [65]:
CLADES = ['S','L','G','V','GR','GH','GV','GK']
y_true = preds.ground_truth
y_pred = preds.pred_class
precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, average=None, labels=CLADES)

In [67]:
list_metrics = []
Metrics = namedtuple("Metrics", ["clade","precision", "recall"])#, "fscore", "support"]
for j,clade in enumerate(CLADES): 
    list_metrics.append(
        Metrics(clade, precision[j], recall[j])
    )

In [68]:
pd.DataFrame(list_metrics)

,clade,precision,recall
0,S,0.999548,0.988606
1,L,0.996724,0.983149
2,G,0.858899,0.963020
3,V,0.992771,0.992771
4,GR,0.997117,0.926960
5,GH,0.992258,0.989413
6,GV,0.997910,0.979688
7,GK,0.965473,0.959468


___
metrics